<a href="https://colab.research.google.com/github/protocol-streams/querent-experimental/blob/main/Full_end_to_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install PyPDF2

In [ ]:
# Connect to Google Colab
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/corpus-data/

Mounted at /content/drive
/content/drive/MyDrive/corpus-data


In [ ]:
#Read the PDF Content



from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    pdf_reader = PdfReader(pdf_path)
    pdf_text = ""

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        pdf_text += page.extract_text()

    return pdf_text

# Replace 'Your_PDF_Name.pdf' with the name of the uploaded PDF
pdf_text = extract_text_from_pdf('eScholarship UC item 4m56569q.pdf')
print(pdf_text[:1000])  # Print the first 1000 characters to check



In [ ]:
#Extract Entities using BERT

from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load pre-trained BERT model and tokenizer
model = BertForTokenClassification.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-large-cased-finetuned-conll03-english')

def extract_entities(text):
    tokens = tokenizer.tokenize(text)
    entities = []

    # Process text in chunks of 510 tokens to account for special tokens
    for i in range(0, len(tokens), 510):
        chunk = tokens[i:i+510]
        inputs = tokenizer.encode(chunk, return_tensors="pt")
        outputs = model(inputs).logits
        predictions = torch.argmax(outputs, dim=2)[0]

        for token, label_id in zip(chunk, predictions[1:-1]):  # Ignore special tokens
            label = model.config.id2label[label_id.item()]
            if label != "O":
                entities.append((token, label))
    return entities

entities = extract_entities(pdf_text)
print(entities[:50])  # Print the first 50 entities to check




Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('UC', 'I-ORG'), ('Irvine', 'I-ORG'), ('UC', 'I-ORG'), ('Irvine', 'I-ORG'), ('Gang', 'I-LOC'), ('##ot', 'I-LOC'), ('##ri', 'I-LOC'), ('Per', 'I-ORG'), ('##mal', 'I-ORG'), ('##ink', 'I-ORG'), ('##hip', 'I-ORG'), ('Journal', 'I-ORG'), ('International', 'I-ORG'), ('Journal', 'I-ORG'), ('of', 'I-ORG'), ('Re', 'I-ORG'), ('##mote', 'I-ORG'), ('Sen', 'I-MISC'), ('##sing', 'I-MISC'), ('Sara', 'I-PER'), ('##s', 'I-PER'), ('##wat', 'I-PER'), ('Pune', 'I-PER'), ('##et', 'I-PER'), ('Syed', 'I-PER'), ('Ta', 'I-PER'), ('##j', 'I-PER'), ('##dar', 'I-PER'), ('##ul', 'I-PER'), ('H', 'I-PER'), ('F', 'I-PER'), ('##ami', 'I-PER'), ('##gli', 'I-PER'), ('##etti', 'I-PER'), ('James', 'I-PER'), ('S', 'I-PER'), ('##S', 'I-ORG'), ('California', 'I-ORG'), ('Digital', 'I-ORG'), ('Library', 'I-ORG'), ('University', 'I-ORG'), ('of', 'I-ORG'), ('California', 'I-ORG'), ('##fo', 'I-ORG'), ('##n', 'I-ORG'), ('##line', 'I-ORG'), ('UC', 'I-ORG'), ('Irvine', 'I-ORG'), ('Libraries', 'I-ORG'), ('International', 'I-ORG')]


In [ ]:
def merge_subwords(entities):
    merged_entities = []
    current_entity = ""
    current_label = ""
    for token, label in entities:
        if token.startswith("##"):
            current_entity += token[2:]
        else:
            if current_entity:
                merged_entities.append((current_entity, current_label))
            current_entity = token
            current_label = label
    if current_entity:
        merged_entities.append((current_entity, current_label))
    return merged_entities

merged_entities = merge_subwords(entities)
print(merged_entities[:50])  # Print the first 50 merged entities to check


[('UC', 'I-ORG'), ('Irvine', 'I-ORG'), ('UC', 'I-ORG'), ('Irvine', 'I-ORG'), ('Gangotri', 'I-LOC'), ('Permalinkhip', 'I-ORG'), ('Journal', 'I-ORG'), ('International', 'I-ORG'), ('Journal', 'I-ORG'), ('of', 'I-ORG'), ('Remote', 'I-ORG'), ('Sensing', 'I-MISC'), ('Saraswat', 'I-PER'), ('Puneet', 'I-PER'), ('Syed', 'I-PER'), ('Tajdarul', 'I-PER'), ('H', 'I-PER'), ('Famiglietti', 'I-PER'), ('James', 'I-PER'), ('SS', 'I-PER'), ('California', 'I-ORG'), ('Digital', 'I-ORG'), ('Library', 'I-ORG'), ('University', 'I-ORG'), ('of', 'I-ORG'), ('Californiafonline', 'I-ORG'), ('UC', 'I-ORG'), ('Irvine', 'I-ORG'), ('Libraries', 'I-ORG'), ('International', 'I-ORG'), ('Journal', 'I-ORG'), ('of', 'I-ORG'), ('Remote', 'I-ORG'), ('Sensingfonline', 'I-MISC'), ('Gangotri', 'I-LOC'), ('Puneet', 'I-PER'), ('Saraswat', 'I-PER'), ('Tajdarul', 'I-PER'), ('H', 'I-PER'), ('.', 'I-PER'), ('Syed', 'I-PER'), ('James', 'I-PER'), ('S', 'I-PER'), ('.', 'I-PER'), ('Famiglietti', 'I-PER'), ('Eric', 'I-PER'), ('J', 'I-PER')

In [ ]:
#Get all entities in an excel file

import pandas as pd
from google.colab import files

# Convert the entities list to a pandas DataFrame
df = pd.DataFrame(merged_entities, columns=['Token', 'Label'])

# Write the DataFrame to an Excel file
df.to_excel('entities.xlsx', index=False)

# Download the file to your local machine
files.download('entities.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Extract Entities using GeoBERT
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("botryan96/GeoBERT")
model = AutoModelForTokenClassification.from_pretrained("botryan96/GeoBERT", from_tf=True)
ner_machine = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

def extract_entities(text):
    # Split the text into chunks of 1000 characters
    text_chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
    entities = []
    for chunk in text_chunks:
        entities += ner_machine(chunk)
    return [(entity['word'], entity['entity_group']) for entity in entities]

entities = extract_entities(pdf_text)
print(entities[:50])  # Print the first 50 entities to check





All TF 2.0 model weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[('velocity', 'GeoMeth'), ('glacier', 'GeoPetro'), ('##56', 'GeoLoc'), ('remote sensing', 'GeoMeth'), ('pune', 'GeoLoc'), ('remote sensing', 'GeoMeth'), ('velocity', 'GeoMeth'), ('glacier', 'GeoPetro'), ('##dar', 'GeoLoc'), ('velocity', 'GeoMeth'), ('glacier', 'GeoLoc'), ('remote sensing', 'GeoMeth'), ('remote sensing', 'GeoMeth'), ('vol', 'GeoMeth'), ('velocity', 'GeoMeth'), ('glacier', 'GeoPetro'), ('##dar', 'GeoLoc'), ('indian', 'GeoPetro'), ('india', 'GeoLoc'), ('geology', 'GeoMeth'), ('indian', 'GeoPetro'), ('india', 'GeoLoc'), ('earth', 'GeoPetro'), ('ir', 'GeoPetro'), ('##vin', 'GeoPetro'), ('usa', 'GeoLoc'), ('hydrologic', 'GeoMeth'), ('modeling', 'GeoMeth'), ('usa', 'GeoLoc'), ('usa', 'GeoLoc'), ('glacier', 'GeoPetro'), ('water cycle', 'GeoMeth'), ('himalayan', 'GeoPetro'), ('glaciers', 'GeoPetro'), ('india', 'GeoLoc'), ('glacier', 'GeoPetro'), ('aperture', 'GeoMeth'), ('radar', 'GeoMeth'), ('offset', 'GeoMeth'), ('thermal', 'GeoMeth'), ('glacier', 'GeoLoc'), ('glacier', 'GeoL

In [ ]:
import pandas as pd

# Convert the entities list to a pandas DataFrame
df = pd.DataFrame(entities, columns=['Entity', 'Label'])

# Write the DataFrame to an Excel file
df.to_excel('entities_GeoBERT.xlsx', index=False)

# Download the file to your local machine
files.download('entities_GeoBERT.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Test the ner_machine pipeline on a small text chunk
test_chunk = pdf_text[:1000]
test_entities = ner_machine(test_chunk)
print(test_entities)

[{'entity_group': 'GeoMeth', 'score': 0.98896956, 'word': 'velocity', 'start': 102, 'end': 110}, {'entity_group': 'GeoPetro', 'score': 0.70181453, 'word': 'glacier', 'start': 123, 'end': 130}, {'entity_group': 'GeoLoc', 'score': 0.49017197, 'word': '##56', 'start': 197, 'end': 199}, {'entity_group': 'GeoMeth', 'score': 0.96155363, 'word': 'remote sensing', 'start': 237, 'end': 251}, {'entity_group': 'GeoLoc', 'score': 0.90600884, 'word': 'pune', 'start': 293, 'end': 297}, {'entity_group': 'GeoMeth', 'score': 0.9892937, 'word': 'remote sensing', 'start': 732, 'end': 746}, {'entity_group': 'GeoMeth', 'score': 0.9860805, 'word': 'velocity', 'start': 895, 'end': 903}, {'entity_group': 'GeoPetro', 'score': 0.7499879, 'word': 'glacier', 'start': 916, 'end': 923}, {'entity_group': 'GeoLoc', 'score': 0.6558047, 'word': '##dar', 'start': 964, 'end': 967}]
